In [7]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import sys

In [60]:
# PROBLEM 1

# PART A
def gram_schmidt(X):
    # X is an n-by-p matrix.
    # Returns U an orthonormal matrix.
    # eps is a threshold value to identify if a vector
    # is nearly a zero vector.
    eps = 1e-12

    n, p = X.shape
    U = np.zeros((n, 0))
    for j in range(p):
        # Get the j-th column of matrix X
        v = X[:, j]

        v = v - U@U.T@v

        if np.linalg.norm(v) > eps:
          u = v / (v@v) ** 0.5

          u = u.reshape(-1,1)

          U = np.append(U, u, 1)
    return U




In [62]:
# PART B

def hilbert_matrix(n):
    X = np.array([[1.0 / (i + j - 1) for i in range(1, n + 1)] for j in range(1, n + 1)])
    return X

hilbert = hilbert_matrix(7)
orthonormal_hilbert = gram_schmidt(hilbert)

error_matrix = np.identity(7) - orthonormal_hilbert.T @ orthonormal_hilbert

error_rate = np.linalg.norm(error_matrix, ord=1)
print(error_rate)


0.2504636824142552


In [10]:
def modified_gram_schmidt(X):
    # Define a threshold value to identify if a vector
    # is nearly a zero vector.
    eps = 1e-12

    n, p = X.shape
    U = np.zeros((n, 0))

    for j in range(p):
        # Get the j-th column of matrix X
        v = X[:, j]
        for i in range(j):
            # Compute and subtract the projection of
            # vector v onto the i-th column of U
            v = v - np.dot(U[:, i], v) * U[:, i]
        v = np.reshape(v, (-1, 1))
        # Check whether the vector we get is nearly
        # a zero vector
        if np.linalg.norm(v) > eps:
            # Normalize vector v and append it to U
            U = np.hstack((U, v / np.linalg.norm(v)))

    return U

In [55]:
# PART C

hilbert = hilbert_matrix(7)
orthonormal_hilbert = modified_gram_schmidt(hilbert)

error_matrix = np.identity(7) - orthonormal_hilbert.T @ orthonormal_hilbert

error_rate = np.linalg.norm(error_matrix, ord=1)
print(error_rate)

7.949993888719218e-09


Here, the modified gram schmidt just subtracts the projections from the prior calculated unit vectors one by one. I'm assuming that doing this step one by one somehow reduces error compared to when the projection of a particular vector v is calculated over the existing basis

In [58]:
# QUESTION 6 PART A AND B

d = np.load('face_emotion_data.npz')
X = d['X']
y = d['y']

n, p = np.shape(X)

# error rate for regularized least squares
error_RLS = np.zeros((8, 7))
# error rate for truncated SVD
error_SVD = np.zeros((8, 7))

# SVD parameters to test
k_vals = np.arange(9) + 1
length_k = len(k_vals)

param_err_SVD = np.zeros(length_k)

# RLS parameters to test
lambda_vals = np.array([0, 0.5, 1, 2, 4, 8, 16])
length_l = len(lambda_vals)

param_err_RLS = np.zeros(length_l)



for h in range (8):
  i_range = np.arange(128)
  holdout_index = np.full(128,False)
  for i in range(16*h,16*(h+1)):
    holdout_index[i] = True

  testing_index = np.logical_not(holdout_index)

  X_train = X[testing_index, :]
  y_train = y[testing_index, :]
  X_holdout = X[holdout_index, :]
  y_holdout = y[holdout_index, :]


  for j in range(7):
    length = X_train.shape[0]
    j_range = np.arange(length)

    true_testing_index = np.full(length,False)
    for k in range(16*j,16*(j+1)):
      true_testing_index[k] = True
    true_holdout_index = np.logical_not(true_testing_index)

    Xh = X_train[true_testing_index, :]
    yh = y_train[true_testing_index, :]
    Xt = X_train[true_holdout_index, :]
    yt = y_train[true_holdout_index, :]


    U, S, Vh = np.linalg.svd(Xt, full_matrices=False)


    # SVD Testing

    w = np.zeros((p, length_k))

    for i in range(length_k):
      k = k_vals[i]

      v_k = Vh[0:k, :]
      s_k = S[0: k]
      u_k = U[:, 0:k]

      w[:, i] = np.dot(v_k.T, np.diag(1 / s_k)).dot(np.dot(u_k.T, yt)).T

      w_reshaped = np.reshape(w[:, i], (p, 1))
      dot_product = np.dot(Xh, w_reshaped)
      yp = np.sign(dot_product)
      param_err_SVD[i] = np.mean(yp != yh)

    param_opt = np.argmin(param_err_SVD)

    # Calculate true error
    param_opt_vector = w[:, param_opt]
    reshaped_param_opt_vector = np.reshape(param_opt_vector, (p, 1))
    dot_product = np.dot(X_holdout, reshaped_param_opt_vector)
    ypp = np.sign(dot_product)
    error_SVD[h, j] = np.mean(ypp != y_holdout)



    # LS

    w = np.zeros((p, length_l))

    for i in range(length_l):
      lam = lambda_vals[i]

      S_scaled = S / (S**2 + lam)
      diagonal_matrix = np.diag(S_scaled)
      first_dot_product = np.dot(Vh.T, diagonal_matrix)
      second_dot_product = np.dot(first_dot_product, np.dot(U.T, yt))
      w[:, i] = second_dot_product.T

      column_vector = w[:, i]
      reshaped_column_vector = np.reshape(column_vector, (p, 1))
      dot_product = np.dot(Xh, reshaped_column_vector)
      yp = np.sign(dot_product)

      param_err_RLS[i] = np.mean(yp != yh)

    param_opt = np.argmin(param_err_RLS)

    # Calculate true error
    column_vector = w[:, param_opt]
    reshaped_column_vector = np.reshape(column_vector, (p, 1))
    dot_product = np.dot(X_holdout, reshaped_column_vector)
    ypp = np.sign(dot_product)
    error_RLS[h, j] = np.mean(ypp != y_holdout)



SVD_error = np.mean(error_SVD)
RLS_error = np.mean(error_RLS)

print(SVD_error)
print(RLS_error)


0.11160714285714286
0.04799107142857143


In [59]:
# QUESTION 6 PART C

d = np.load('face_emotion_data.npz')
X = d['X']

augmented_features = X @ np.random.rand(9,3)
X = np.hstack((X, augmented_features))
y = d['y']

n, p = np.shape(X)

# error rate for regularized least squares
error_RLS = np.zeros((8, 7))
# error rate for truncated SVD
error_SVD = np.zeros((8, 7))

# SVD parameters to test
k_vals = np.arange(9) + 1
length_k = len(k_vals)

param_err_SVD = np.zeros(length_k)

# RLS parameters to test
lambda_vals = np.array([0, 0.5, 1, 2, 4, 8, 16])
length_l = len(lambda_vals)

param_err_RLS = np.zeros(length_l)



for h in range (8):
  i_range = np.arange(128)
  holdout_index = np.full(128,False)
  for i in range(16*h,16*(h+1)):
    holdout_index[i] = True

  testing_index = np.logical_not(holdout_index)

  X_train = X[testing_index, :]
  y_train = y[testing_index, :]
  X_holdout = X[holdout_index, :]
  y_holdout = y[holdout_index, :]


  for j in range(7):
    length = X_train.shape[0]
    j_range = np.arange(length)

    true_testing_index = np.full(length,False)
    for k in range(16*j,16*(j+1)):
      true_testing_index[k] = True
    true_holdout_index = np.logical_not(true_testing_index)

    Xh = X_train[true_testing_index, :]
    yh = y_train[true_testing_index, :]
    Xt = X_train[true_holdout_index, :]
    yt = y_train[true_holdout_index, :]


    U, S, Vh = np.linalg.svd(Xt, full_matrices=False)


    # SVD Testing

    w = np.zeros((p, length_k))

    for i in range(length_k):
      k = k_vals[i]

      v_k = Vh[0:k, :]
      s_k = S[0: k]
      u_k = U[:, 0:k]

      w[:, i] = np.dot(v_k.T, np.diag(1 / s_k)).dot(np.dot(u_k.T, yt)).T

      w_reshaped = np.reshape(w[:, i], (p, 1))
      dot_product = np.dot(Xh, w_reshaped)
      yp = np.sign(dot_product)
      param_err_SVD[i] = np.mean(yp != yh)

    param_opt = np.argmin(param_err_SVD)

    # Calculate true error
    param_opt_vector = w[:, param_opt]
    reshaped_param_opt_vector = np.reshape(param_opt_vector, (p, 1))
    dot_product = np.dot(X_holdout, reshaped_param_opt_vector)
    ypp = np.sign(dot_product)
    error_SVD[h, j] = np.mean(ypp != y_holdout)



    # LS

    w = np.zeros((p, length_l))

    for i in range(length_l):
      lam = lambda_vals[i]

      S_scaled = S / (S**2 + lam)
      diagonal_matrix = np.diag(S_scaled)
      first_dot_product = np.dot(Vh.T, diagonal_matrix)
      second_dot_product = np.dot(first_dot_product, np.dot(U.T, yt))
      w[:, i] = second_dot_product.T

      column_vector = w[:, i]
      reshaped_column_vector = np.reshape(column_vector, (p, 1))
      dot_product = np.dot(Xh, reshaped_column_vector)
      yp = np.sign(dot_product)

      param_err_RLS[i] = np.mean(yp != yh)

    param_opt = np.argmin(param_err_RLS)

    # Calculate true error
    column_vector = w[:, param_opt]
    reshaped_column_vector = np.reshape(column_vector, (p, 1))
    dot_product = np.dot(X_holdout, reshaped_column_vector)
    ypp = np.sign(dot_product)
    error_RLS[h, j] = np.mean(ypp != y_holdout)



SVD_error = np.mean(error_SVD)
RLS_error = np.mean(error_RLS)

print(SVD_error)
print(RLS_error)


0.10825892857142858
0.052455357142857144


For Part C, after running the simulation several times, it seems that adding on these features doesn't really improve the performance. This makes sense as they are merely linear representations of existing features, there is no additional information. So, the algorithm is not training on any new data.